## Build Functions for ELT

In [2]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 uninstall pandas pyarrow fireducks numpy -y
# !pip3 install -r requirements.txt

In [3]:

import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [4]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [5]:
f.predictions('nvda')

NVDA 5m Interval Timestamp: 2024-11-29 05:51:34 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.371591,0.368737,0.345201,0.189037
prob_up,0.236798,0.237618,0.308173,0.25
prob_static,0.449056,0.429677,0.362969,0.625
prob_down,0.314146,0.332705,0.328858,0.125
precision,0.551136,0.549098,0.532286,0.420741
recall,0.570588,0.537255,0.598039,0.556863
f1,0.560694,0.543112,0.56325,0.479325
support,"[509.0, 510.0, 509.0]","[509.0, 510.0, 509.0]","[509.0, 510.0, 509.0]","[509.0, 510.0, 509.0]"


NVDA 15m Interval Timestamp: 2024-11-29 05:51:42 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.319774,0.256983,0.27929,0.183333
prob_up,0.228604,0.268303,0.265477,0.0
prob_static,0.495649,0.465531,0.407676,0.5
prob_down,0.275747,0.266166,0.326847,0.5
precision,0.514124,0.469274,0.485207,0.416667
recall,0.535294,0.494118,0.482353,0.529412
f1,0.524496,0.481375,0.483776,0.466321
support,"[168.0, 170.0, 168.0]","[168.0, 170.0, 168.0]","[168.0, 170.0, 168.0]","[168.0, 170.0, 168.0]"


NVDA 1h Interval Timestamp: 2024-11-29 05:51:45 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,down,static,static
kelly_1:2.5,0.109895,0.070769,0.126027,0.058662
prob_up,0.330185,0.337021,0.338152,0.25
prob_static,0.393017,0.306266,0.343966,0.375
prob_down,0.276798,0.356713,0.317882,0.375
precision,0.364211,0.336264,0.375734,0.327616
recall,0.399538,0.354988,0.443418,0.441109
f1,0.381057,0.345372,0.40678,0.375984
support,"[432.0, 433.0, 431.0]","[432.0, 433.0, 431.0]","[432.0, 433.0, 431.0]","[432.0, 433.0, 431.0]"


NVDA 1wk Interval Timestamp: 2024-11-29 05:51:57 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.151515,0.048544,0.101667,0.041441
prob_up,0.12028,0.25466,0.258418,0.125
prob_static,0.762173,0.491929,0.480795,0.5
prob_down,0.117547,0.253412,0.260787,0.375
precision,0.393939,0.320388,0.358333,0.315315
recall,0.443182,0.375,0.488636,0.397727
f1,0.417112,0.34555,0.413462,0.351759
support,"[88.0, 88.0, 87.0]","[88.0, 88.0, 87.0]","[88.0, 88.0, 87.0]","[88.0, 88.0, 87.0]"


NVDA 1mo Interval Timestamp: 2024-11-29 05:52:00 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,static
kelly_1:2.5,0.172727,-0.2,-0.213333,0.125
prob_up,0.517707,0.33657,0.355946,0.125
prob_static,0.289652,0.327857,0.338716,0.5
prob_down,0.192642,0.335573,0.305339,0.375
precision,0.409091,0.142857,0.133333,0.375
recall,0.5,0.111111,0.111111,0.45
f1,0.45,0.125,0.121212,0.409091
support,"[18.0, 20.0, 19.0]","[18.0, 20.0, 19.0]","[18.0, 20.0, 19.0]","[18.0, 20.0, 19.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')